In [ ]:
# import all Imp Libraries
import pandas as pd
import numpy as np
import re
import string
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [ ]:
# punt,stopwards,wordnet
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
df=pd.read_csv('/content/sentiment_dataset_raw.csv')

In [ ]:
df

,Review,sentiment
0,I absolutely loved this movie! 😊 😍,positive
1,"Terrible experience, would not recommend. !!",negative
2,I absolutely loved this movie! 😊 😍,positive
3,"Had a great time with friends, everything was ...",positive
4,I absolutely loved this movie! 😊,positive
...,...,...
5995,What a fantastic experience. Will come again! !!!,positive
5996,Worst product ever. Total waste of money!,negative
5997,"Had a great time with friends, everything was ...",positive
5998,"The service was excellent, highly recommend!",positive


# Lowercasing:
  convert all the sentence words into lowerwords

In [ ]:
df['Review']=df['Review'].str.lower()

Remove HTML tags

In [ ]:
df[df['Review'].str.contains('<')]

,Review,sentiment
9,"superb product quality, very satisfied. <br>",positive
12,i hated every minute of it. 😡 <br>,negative
13,not worth the time or the price. <br>,negative
22,"terrible experience, would not recommend. <br>",negative
34,"terrible experience, would not recommend. <br>",negative
...,...,...
5970,i'm very disappointed with the service. <br>,negative
5973,"had a great time with friends, everything was ...",positive
5985,i absolutely loved this movie! 😊 <br>,positive
5987,"terrible experience, would not recommend. <br>",negative


In [ ]:
df['Review']=df['Review'].apply(lambda x:BeautifulSoup(x,'html.parser').get_text())

In [ ]:
df[df['Review'].str.contains('<')]

,Review,sentiment


CHECKING and REMOVING URLs

In [ ]:
df[df['Review'].str.contains('http')]


,Review,sentiment
5,"the service was excellent, highly recommend! h...",positive
14,"superb product quality, very satisfied. http:/...",positive
24,i hated every minute of it. 😡 http://example.com,negative
41,"superb product quality, very satisfied. http:/...",positive
50,not worth the time or the price. http://exampl...,negative
...,...,...
5959,i hated every minute of it. 😡 http://example.com,negative
5978,"had a great time with friends, everything was ...",positive
5980,i'm very disappointed with the service. http:/...,negative
5981,i hated every minute of it. 😡 http://example.com,negative


In [ ]:
df['Review']=df['Review'].apply(lambda x:re.sub(r'http\S+','',x))

# Remove Punctuations

In [ ]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
z=lambda x: x.translate(str.maketrans('','',string.punctuation))

In [ ]:
z('dhsefjebs34r6#%^&')

'dhsefjebs34r6'

In [ ]:
df['Review']=df['Review'].apply(z)

Chat word Treatment


Replace slag/ Abbreviations(eg.'u'--->'you')
convert informal text to standard text

In [ ]:
chat_word_map={
    'u':'you',
    'ur':'your',
    'lol':'laughing out loud',
    'omg':'oh my god',
    'idk':'i do not know',
    'btw':'by the way',
    'brb':'be right back'
}

In [ ]:
def expand_chat_word(text):
  return ' '.join([chat_word_map[word] if word in chat_word_map else word for word in text.split()])

In [ ]:
df['Review'].apply(expand_chat_word)

,Review
0,i absolutely loved this movie 😊 😍
1,terrible experience would not recommend
2,i absolutely loved this movie 😊 😍
3,had a great time with friends everything was p...
4,i absolutely loved this movie 😊
...,...
5995,what a fantastic experience will come again
5996,worst product ever total waste of money
5997,had a great time with friends everything was p...
5998,the service was excellent highly recommend


Speling Correction(Optional But slow)
correct missplaced words
use tool >> Textblob, SymSpell or autocorrect

In [ ]:
from textblob import TextBlob
df['Review'].apply(lambda x: str(TextBlob(x).correct()))

,Review
0,i absolutely loved this movie 😊 😍
1,terrible experience would not recommend
2,i absolutely loved this movie 😊 😍
3,had a great time with friends everything was p...
4,i absolutely loved this movie 😊
...,...
5995,what a fantastic experience will come again
5996,worst product ever total waste of money
5997,had a great time with friends everything was p...
5998,the service was excellent highly recommend


# Stop word Remove
  Remove common word like 'The', 'Is','and'.

In [ ]:
nltk.download('stopwords')
stop_word=set(stopwords.words('english'))
print(stop_word)

{"should've", "didn't", 'its', 'own', 'further', 'm', 't', "she'd", 'no', "haven't", "wasn't", 'their', 'or', 'against', 'because', 'in', 'both', 'weren', 'mightn', 'ours', 'be', 'needn', 'themselves', 'same', 'of', "couldn't", 'mustn', 's', 'why', 're', 'are', "we'd", 'through', 'ma', 'hadn', 'that', "that'll", 'as', "won't", 'his', "i'd", 'can', "they've", 'should', 'up', 'into', 'herself', 'y', 'who', 'yours', 'd', 'me', 'so', 'didn', "mightn't", 'won', 'each', 'again', 'we', 'too', 'during', 'about', 'the', 'to', 'a', 'did', "he's", 'wouldn', 'for', 'does', 'with', 'above', 'your', 'you', 'there', 'where', 'our', "they'll", "we've", 'is', 'out', 'yourself', 'my', "we're", 'by', 'an', 'll', 'had', 'him', 'not', "you'd", "it'll", "you've", 'her', 'o', 'don', 'before', 'once', 'being', "you'll", 'then', 'this', 'below', 'i', 'has', 'any', 'under', 'here', 'on', "they're", 'while', 'and', "hasn't", 'all', "doesn't", 'having', "don't", 'some', 'now', 'was', 'yourselves', 'at', "weren't"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.tokenize import word_tokenize
def remove_stop_words(text):
  tokens=word_tokenize(text)
  return ' '.join([word for word in tokens if word not in stop_word])

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import nltk
nltk.download('punk_tab')
df['Review']=df['Review'].apply(remove_stop_words)
df

[nltk_data] Error loading punk_tab: Package 'punk_tab' not found in
[nltk_data]     index


,Review,sentiment
0,absolutely loved movie 😊 😍,positive
1,terrible experience would recommend,negative
2,absolutely loved movie 😊 😍,positive
3,great time friends everything perfect lol,positive
4,absolutely loved movie 😊,positive
...,...,...
5995,fantastic experience come,positive
5996,worst product ever total waste money,negative
5997,great time friends everything perfect,positive
5998,service excellent highly recommend,positive


# Handling emojis
  convert emotion images into text

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 12.6 MB/s eta 0:00:00


In [ ]:
import emoji
emoji.demojize('😊')

':smiling_face_with_smiling_eyes:'

In [ ]:
df['Review'].apply(lambda x: emoji.demojize(x,delimiters=('','')))

,Review
0,absolutely loved movie smiling_face_with_smili...
1,terrible experience would recommend
2,absolutely loved movie smiling_face_with_smili...
3,great time friends everything perfect lol
4,absolutely loved movie smiling_face_with_smili...
...,...
5995,fantastic experience come
5996,worst product ever total waste money
5997,great time friends everything perfect
5998,service excellent highly recommend


# Tokenization
  split sentences into words

In [ ]:
df['Tokenize Review']=df['Review'].apply(word_tokenize)
df

,Review,sentiment,Tokenize Review
0,absolutely loved movie 😊 😍,positive,"[absolutely, loved, movie, 😊, 😍]"
1,terrible experience would recommend,negative,"[terrible, experience, would, recommend]"
2,absolutely loved movie 😊 😍,positive,"[absolutely, loved, movie, 😊, 😍]"
3,great time friends everything perfect lol,positive,"[great, time, friends, everything, perfect, lol]"
4,absolutely loved movie 😊,positive,"[absolutely, loved, movie, 😊]"
...,...,...,...
5995,fantastic experience come,positive,"[fantastic, experience, come]"
5996,worst product ever total waste money,negative,"[worst, product, ever, total, waste, money]"
5997,great time friends everything perfect,positive,"[great, time, friends, everything, perfect]"
5998,service excellent highly recommend,positive,"[service, excellent, highly, recommend]"


# Stemming:
  Reduce words to base / root (e.g. 'Playing ====> 'play' =====> "play",played-----> 'play')

In [ ]:
stemmer=PorterStemmer()
def steam_word(text):
  return[stemmer.stem(word) for word in text]
df['Clean_Review']=df['Tokenize Review'].apply(steam_word)
df

,Review,sentiment,Tokenize Review,Clean_Review
0,absolutely loved movie 😊 😍,positive,"[absolutely, loved, movie, 😊, 😍]","[absolut, love, movi, 😊, 😍]"
1,terrible experience would recommend,negative,"[terrible, experience, would, recommend]","[terribl, experi, would, recommend]"
2,absolutely loved movie 😊 😍,positive,"[absolutely, loved, movie, 😊, 😍]","[absolut, love, movi, 😊, 😍]"
3,great time friends everything perfect lol,positive,"[great, time, friends, everything, perfect, lol]","[great, time, friend, everyth, perfect, lol]"
4,absolutely loved movie 😊,positive,"[absolutely, loved, movie, 😊]","[absolut, love, movi, 😊]"
...,...,...,...,...
5995,fantastic experience come,positive,"[fantastic, experience, come]","[fantast, experi, come]"
5996,worst product ever total waste money,negative,"[worst, product, ever, total, waste, money]","[worst, product, ever, total, wast, money]"
5997,great time friends everything perfect,positive,"[great, time, friends, everything, perfect]","[great, time, friend, everyth, perfect]"
5998,service excellent highly recommend,positive,"[service, excellent, highly, recommend]","[servic, excel, highli, recommend]"


# Lemmatization :
  Convert words to dist form e.g ('better' ----> 'goog')

In [ ]:
lemmatizer=WordNetLemmatizer()
def lemmatizer_word(text):
  return [lemmatizer.lemmatize(word) for word in text]

In [ ]:
df['Clean_Review']=df['Clean_Review'].apply(lemmatizer_word)

# BOW : Vectorization
  

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
df['string_review']=df['Clean_Review'].apply(lambda x:' '.join(x))
vectorizer=CountVectorizer()
bow=vectorizer.fit_transform(df['string_review'])
print(bow)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 23634 stored elements and shape (6000, 34)>
  Coords	Values
  (0, 0)	1
  (0, 15)	1
  (0, 18)	1
  (1, 27)	1
  (1, 7)	1
  (1, 33)	1
  (1, 23)	1
  (2, 0)	1
  (2, 15)	1
  (2, 18)	1
  (3, 10)	1
  (3, 28)	1
  (3, 9)	1
  (3, 5)	1
  (3, 19)	1
  (3, 14)	1
  (4, 0)	1
  (4, 15)	1
  (4, 18)	1
  (5, 23)	1
  (5, 25)	1
  (5, 6)	1
  (5, 12)	1
  (6, 10)	1
  (6, 28)	1
  :	:
  (5994, 1)	1
  (5995, 7)	1
  (5995, 8)	1
  (5995, 1)	1
  (5996, 21)	1
  (5996, 31)	1
  (5996, 3)	1
  (5996, 29)	1
  (5996, 30)	1
  (5996, 17)	1
  (5997, 10)	1
  (5997, 28)	1
  (5997, 9)	1
  (5997, 5)	1
  (5997, 19)	1
  (5998, 23)	1
  (5998, 25)	1
  (5998, 6)	1
  (5998, 12)	1
  (5999, 21)	1
  (5999, 31)	1
  (5999, 3)	1
  (5999, 29)	1
  (5999, 30)	1
  (5999, 17)	1


In [ ]:
vectorizer.vocabulary_
bow[0].toarray()

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
n_gram=vectorizer.fit_transform(df['string_review'])
n_gram[0].toarray()

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
n_gram.shape

(6000, 34)

In [ ]:
bow.shape

(6000, 34)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
x_tfidf=vectorizer.fit_transform(df['string_review'])


In [ ]:
x_tfidf.shape

(6000, 34)

In [ ]:
from scipy.sparse import csr_matrix
x=x_tfidf
# store each row as a dense numpy array in a list
feature_list=[x[i].toarray().flatten() for i in range(x.shape[0])]
feature_list

[array([0.57735027, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.57735027, 0.        , 0.        , 0.57735027, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]),
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.44215901, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.43667923, 0.        ,
        0.        , 0.        , 0.55398857, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.55398857]),
 array([0.57735027, 0.        , 0.        , 0.        , 0.        ,


In [ ]:
# or if you want to store as sparse vector (for storage efficiency)
sparse_feature_list=[x[i] for i in range(x.shape[0])]


In [ ]:
from scipy.sparse import save_npz
save_npz('tfidf_Matrix.npz',x)  # why x beacause ---x is our csr_matrix

In [ ]:
from scipy.sparse import load_npz
x=load_npz('tfidf_Matrix.npz')

In [ ]:
y=df['sentiment']
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(y)
y

array([1, 0, 1, ..., 1, 1, 0])

In [ ]:
# import all imortant libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.preprocessing import StandardScaler


In [ ]:
# convert tfidf sparse matrix to array
x_array=x.toarray()

In [ ]:
# Train test split
x_train,x_test,y_train,y_test=train_test_split(x_array,y,test_size=0.2,random_state=42)

In [ ]:
# train the model
model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [ ]:
y_pred=model.predict(x_test)
y_pred

array([1, 0, 1, ..., 1, 0, 1])

In [ ]:
accuracy_score(y_test,y_pred)


1.0

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
accuracy_score(y_train,y_pred)

1.0

In [ ]:
y_pred_test=model.predict(x_test)
print(classification_report(y_test,y_pred_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       571
           1       1.00      1.00      1.00       629

    accuracy                           1.00      1200
   macro avg       1.00      1.00      1.00      1200
weighted avg       1.00      1.00      1.00      1200

